In [47]:
import pandas as pd
import numpy as np

In [48]:
# definir arquivo Pickle de entrada
arquivo = 'afastamentos-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'cadastro-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'extra/nome_sexo_pf05.csv'

df = pd.read_pickle(arquivo)
df.head()

,SEXO,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
PNOME,,,,,,,,,,,,,,,,,
MARIA,0,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,1,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANTONIO,1,15,9,14,15,20,14,1,0,0,0,0,0,0,0,0,0
FRANCISCO,1,15,3,19,9,3,14,1,18,6,0,0,0,0,0,0,0
JOAO,1,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0


In [49]:
df.shape

(9726, 17)

In [50]:
df.shape

(9726, 17)

### Decomposição com PCA

In [51]:
#from sklearn.decomposition import PCA
#X_reduced = PCA(n_components=4).fit_transform(df.drop(['SEXO'], axis=1))
#X_reduced[:5]

In [52]:
#X = pd.DataFrame(X_reduced, index=df.index)
#y = df['SEXO'] # apenas a coluna alvo
#print('Forma dos dados:', X.shape, y.shape)

### Uso dos dados originais

In [53]:
X = df.drop(['SEXO'], axis=1) # tudo, exceto a coluna alvo
y = df['SEXO'] # apenas a coluna alvo
print('Forma dos dados:', X.shape, y.shape)

Forma dos dados: (9726, 16) (9726,)


In [54]:
X.head()

,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
PNOME,,,,,,,,,,,,,,,,
MARIA,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANTONIO,15,9,14,15,20,14,1,0,0,0,0,0,0,0,0,0
FRANCISCO,15,3,19,9,3,14,1,18,6,0,0,0,0,0,0,0
JOAO,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0


In [55]:
y.head()

PNOME
MARIA        0
JOSE         1
ANTONIO      1
FRANCISCO    1
JOAO         1
Name: SEXO, dtype: uint8

In [56]:
NUMBER_KFOLD_SPLITS = 5 # number of splits in cross-validation
NUMBER_GRID_ITERATIONS = 10 # number of grid iterations to parameters testing
SCORING_METRIC = 'accuracy' # the scoring metric to be used

In [57]:
from datetime import datetime
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

models = {}

def evaluate_model(name, model, X=X, y=y):

  start = datetime.now()
  kfold = KFold(n_splits=NUMBER_KFOLD_SPLITS, shuffle=True, random_state=42)
  results = cross_val_score(model, X, y, cv=kfold,
                            scoring=SCORING_METRIC, verbose=1, n_jobs=-1)
  end = datetime.now()

  elapsed = int((end - start).total_seconds() * 1000)
  score = results.mean() * 100
  stddev = results.std() * 100

  models[name] = (model, score, stddev, elapsed)
  print(model, '\nCross-Validation Score: %.2f (+/- %.2f) [%5s ms]' % \
        (score, stddev, elapsed))
  return score, stddev, elapsed

In [58]:
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
#from sklearn.model_selection import GridSearchCV

# faz o ajuste fino do modelo, calculando os melhores hiperparâmetros
def fine_tune_model(model, params, X=X, y=y):

  print('\nFine Tuning Model:')
  print(model, "\nparams:", params)
  
  kfold = KFold(n_splits=NUMBER_KFOLD_SPLITS, shuffle=True, random_state=42)

  search = RandomizedSearchCV(model, param_distributions=params, 
                              n_iter=NUMBER_GRID_ITERATIONS, 
                              scoring=SCORING_METRIC, cv=kfold, 
                              verbose=1, n_jobs=-1)
  #search = GridSearchCV(estimator=model, param_grid=params, scoring='accuracy', cv=kfold, verbose=1)

  search.fit(X, y)
  print('\nBest Score: %.2f %%' % (search.best_score_ * 100))
  print('Best Params:', search.best_params_)
  return search

In [59]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(
    random_state=42, multi_class='auto', 
    max_iter=1000, solver='liblinear', C=1.0) # 85.18
evaluate_model('LR', model)

params = dict(
    solver=['liblinear','lbfgs'], 
    C=np.logspace(-3,3,7)
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LogisticRegression(max_iter=1000, random_state=42, solver='liblinear') 
Cross-Validation Score: 85.27 (+/- 0.70) [ 2640 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.6s finished


In [60]:
# Linear SVM
from sklearn.svm import LinearSVC

model = LinearSVC(random_state=42, max_iter=1000, C=0.001) # 85.27
evaluate_model('LSVM', model)

#from scipy.stats import reciprocal, uniform
#params = dict(C=uniform(1, 10))
params = dict(
    C=np.logspace(-4,3,8)
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LinearSVC(C=0.001, random_state=42) 
Cross-Validation Score: 85.16 (+/- 0.68) [  712 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.7s finished


In [61]:
# K-Nearest Neighbours (KNN)
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=11) # 87.45
evaluate_model('KNN', model)

params = dict(
    n_neighbors=[1,3,5,7,9,11]
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


KNeighborsClassifier(n_neighbors=11) 
Cross-Validation Score: 87.38 (+/- 0.43) [ 2702 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.7s finished


In [62]:
# Linear Discriminant Analysis (LDA)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

model = LinearDiscriminantAnalysis(solver='svd') # 84.45
evaluate_model('LDA', model)

params = dict(
    solver=['svd','lsqr','eigen']
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LinearDiscriminantAnalysis() 
Cross-Validation Score: 85.18 (+/- 0.64) [  212 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished


In [63]:
# Quadratic Discriminant Analysis (QDA)
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

model = QuadraticDiscriminantAnalysis() # 44.99
evaluate_model('QDA', model)

params = dict()
#fine_tune_model(model, params)

QuadraticDiscriminantAnalysis() 
Cross-Validation Score: 44.99 (+/- 0.74) [  149 ms]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished


In [64]:
# Naïve Bayes - Gaussian
from sklearn.naive_bayes import GaussianNB

model = GaussianNB(priors=None, var_smoothing=1.0) # 67.62
evaluate_model('GNB', model)

params = dict(
    var_smoothing=np.logspace(0,-9,num=10)
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


GaussianNB(var_smoothing=1.0) 
Cross-Validation Score: 85.29 (+/- 0.68) [  116 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished


In [65]:
# Naïve Bayes - Bernoulli
from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB() # 43.37
evaluate_model('BNB', model)

#params = dict(var_smoothing=np.logspace(0,-9,num=10))
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


BernoulliNB() 
Cross-Validation Score: 44.21 (+/- 0.56) [  120 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished


(44.21145213832606, 0.5612128048081376, 120)

In [66]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(
    random_state=42, criterion='entropy', max_depth=7) # 96.37
evaluate_model('DT', model)

params = dict(
    criterion=['gini','entropy'],
    max_depth=[3,5,7,11,13,17,19]
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


DecisionTreeClassifier(criterion='entropy', max_depth=7, random_state=42) 
Cross-Validation Score: 94.04 (+/- 0.31) [  171 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished


In [67]:
# Ensemble - Random Forest
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    random_state=42, max_features='sqrt', n_estimators=500) # 93.86
evaluate_model('RF', model)

params = dict(
    n_estimators=[10,50,100,500], 
    max_features=['auto','sqrt','log2']
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


RandomForestClassifier(max_features='sqrt', n_estimators=500, random_state=42) 
Cross-Validation Score: 93.43 (+/- 0.63) [19543 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   19.5s finished


In [68]:
# Ensemble - Ada Boost
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(
    DecisionTreeClassifier(random_state=42, criterion='entropy', max_depth=17), 
    n_estimators=11) # 96.41
evaluate_model('ABDT', model)

params = dict(
    n_estimators=[1,3,5,7,9,11,13]
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


AdaBoostClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                         max_depth=17,
                                                         random_state=42),
                   n_estimators=11) 
Cross-Validation Score: 94.00 (+/- 0.69) [ 1208 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.2s finished


In [69]:
# Stochastic Gradient Descent (SGD)
from sklearn.linear_model import SGDClassifier

model = SGDClassifier(random_state=42, max_iter=350, tol=0.01) # 77.02
evaluate_model('SGD', model)

params = dict(
    max_iter=[100, 200, 350, 500, 1000], 
    tol=[0.01, 0.1, 1.0]
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


SGDClassifier(max_iter=350, random_state=42, tol=0.01) 
Cross-Validation Score: 72.87 (+/- 13.96) [ 2027 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.0s finished


In [70]:
# Perceptron
from sklearn.linear_model import Perceptron

model = Perceptron(random_state=42, max_iter=500, tol=0.1) # 75.58
evaluate_model('PCT', model)

params = dict(
    max_iter=[100, 200, 350, 500, 750, 1000], 
    tol=[0.1, 0.01, 0.001]
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Perceptron(max_iter=500, random_state=42, tol=0.1) 
Cross-Validation Score: 72.31 (+/- 9.29) [  320 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.3s finished


In [71]:
# Extreme Gradient Boosting Machine (XGBM)
from xgboost import XGBClassifier

model = XGBClassifier(
    max_depth=3, min_child_weight=3, gamma=0.4, 
    subsample=0.7, colsample_bytree=0.8, reg_alpha=1e-05) # 97.67
evaluate_model('XGB', model)

params = dict(max_depth=range(3,10,2), min_child_weight=range(1,6,2)) # 97.60
#params = dict(gamma=[i/10.0 for i in range(0,5)]) # 97.68
#params = dict(subsample=[i/10.0 for i in range(6,10)], colsample_bytree=[i/10.0 for i in range(6,10)]) # 97.72
#params = dict(reg_alpha=[1e-5, 1e-2, 0.1, 1, 100]) # 97.74
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.8, gamma=0.4,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=3,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=1e-05, reg_lambda=None,
              scale_pos_weight=None, subsample=0.7, tree_method=None,
              validate_parameters=None, verbosity=None) 
Cross-Validation Score: 94.76 (+/- 0.85) [ 9545 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    9.5s finished


In [72]:
# LightGBM
from lightgbm import LGBMClassifier
#!pip install lightgbm

model = LGBMClassifier(
    num_leaves=20, max_depth=3, 
    learning_rate=0.1, feature_fraction=1.0) # 97.51
evaluate_model('LGBM', model)

params = dict(num_leaves=range(10,100), max_depth=range(1,30)) # 97.18
#params = dict(learning_rate=[0.01,0.05,0.1,0.5]) # 97.51
#params = dict(feature_fraction=[0.1,0.25,0.5,0.75,1.0]) # 97.62
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LGBMClassifier(feature_fraction=1.0, max_depth=3, num_leaves=20) 
Cross-Validation Score: 94.40 (+/- 0.49) [ 3208 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.2s finished


In [73]:
# CatBoost
from catboost import CatBoostClassifier
#!pip install catboost

model = CatBoostClassifier(
    silent=True, iterations=250,
    learning_rate=0.05, depth=7) # 97.04
#learning_rate=0.1, iterations=1000, depth=5
evaluate_model('CB', model)

params = dict(
    iterations=[10,50,100,250,500,1000],
    learning_rate=[0.01,0.05,0.1,0.5],
    depth=range(1,11,2)
) 
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Cross-Validation Score: 94.49 (+/- 0.49) [12168 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   12.2s finished


In [74]:
names = []
estimators = []
scores = []
stdevs = []
times = []

for key, value in models.items():
    (model, score, stdev, elapsed) = value
    names.append(key)
    estimators.append(model)
    scores.append(score)
    stdevs.append(stdev)
    times.append(elapsed)

In [75]:
results_df = pd.DataFrame({
    'Model': names,
    'Score': scores,
    'Std Dev': stdevs,
    'Time (ms)': times,
    'Estimator': estimators})

results_df.sort_values(by='Score', ascending=False)

,Model,Score,Std Dev,Time (ms),Estimator
12,XGB,94.756170,0.851727,9545,"XGBClassifier(base_score=None, booster=None, c..."
14,CB,94.488944,0.485484,12168,<catboost.core.CatBoostClassifier object at 0x...
13,LGBM,94.396410,0.490468,3208,"LGBMClassifier(feature_fraction=1.0, max_depth..."
7,DT,94.036597,0.311995,171,"DecisionTreeClassifier(criterion='entropy', ma..."
9,ABDT,93.995572,0.694479,1208,AdaBoostClassifier(base_estimator=DecisionTree...
8,RF,93.429924,0.625520,19543,"RandomForestClassifier(max_features='sqrt', n_..."
2,KNN,87.384296,0.425702,2702,KNeighborsClassifier(n_neighbors=11)
5,GNB,85.286784,0.682386,116,GaussianNB(var_smoothing=1.0)
0,LR,85.266208,0.696586,2640,"LogisticRegression(max_iter=1000, random_state..."
3,LDA,85.183983,0.644626,212,LinearDiscriminantAnalysis()


In [76]:
model = models['XGB'][0]
model

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.8, gamma=0.4,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=3,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=1e-05, reg_lambda=None,
              scale_pos_weight=None, subsample=0.7, tree_method=None,
              validate_parameters=None, verbosity=None)

In [77]:
# treinar o melhor modelo com todos os dados
model.fit(X, y)

/home/rodrigo/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:21:21] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.4, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=3,
              min_child_weight=3, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=2, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=1e-05,
              reg_lambda=1, scale_pos_weight=None, subsample=0.7,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [78]:
# definir arquivo Pickle de entrada
#arquivo = 'afastamentos-nomes-sexo-preparado.pkl.bz2'
arquivo = 'cadastro-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'extra/nome_sexo_pf05.csv'

df2 = pd.read_pickle(arquivo)
df2.head()

,SEXO,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
PNOME,,,,,,,,,,,,,,,,,
MARIA,0,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,1,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANA,0,1,14,1,0,0,0,0,0,0,0,0,0,0,0,0,0
JOAO,1,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0
CARLOS,1,19,15,12,18,1,3,0,0,0,0,0,0,0,0,0,0


In [79]:
df2.shape

(48970, 17)

In [80]:
X_test = df2.drop(['SEXO'], axis=1) # tudo, exceto a coluna alvo
y_test = df2['SEXO'] # apenas a coluna alvo

In [81]:
y_pred = model.predict(X_test)
y_pred

array([0, 0, 0, ..., 1, 1, 1], dtype=uint8)

In [82]:
from sklearn.metrics import accuracy_score

print("Accuracy score: %.2f %%" % (accuracy_score(y_test, y_pred) * 100))

Accuracy score: 94.77 %


In [83]:
from sklearn.metrics import confusion_matrix

print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

Confusion matrix:
 [[21572   146   173]
 [   60 18915   284]
 [ 1021   878  5921]]


In [84]:
dados = pd.DataFrame({'REAL': y_test, 'PREV': y_pred}, index=X_test.index)
for col in dados.columns:
    dados[col] = dados[col].map({0: 'F', 1: 'M', 9: 'X'})
dados.head()

,REAL,PREV
PNOME,,
MARIA,F,F
JOSE,M,F
ANA,F,F
JOAO,M,M
CARLOS,M,M


In [85]:
dados[dados['REAL'] != dados['PREV']].head(20)

,REAL,PREV
PNOME,,
JOSE,M,F
LUIZ,M,F
LUIS,M,X
JORGE,M,X
GUILHERME,M,X
RAQUEL,F,M
THAIS,F,X
BEATRIZ,F,X
LILIAN,F,M
